In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import torch

In [2]:
# Carregar modelo e tokenizador
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-llm-7b-chat")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/deepseek-llm-7b-chat",
    #Auto-alocação entre dispositivos
    device_map = "auto",
    torch_dtype=torch.float16,  # Usa meia precisão
    low_cpu_mem_usage=True  # Reduz uso de RAM
)

embeddings = HuggingFaceEmbeddings(
    model_name="neuralmind/bert-base-portuguese-cased"
)

vetorAnimes = FAISS.load_local('C:/Repositorios/RAG-Anime/one_piece_faiss_index/',embeddings,allow_dangerous_deserialization=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_6816\353247676.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
No sentence-transformers model found with name neuralmind/bert-base-portuguese-cased. Creating a new one with mean pooling.


In [3]:
# Função para gerar respostas
def gerar_respostaComVetorAnime(query, k=2):
   
    docAnimes = vetorAnimes.similarity_search(query, k=k)

    context1 = "\n".join([doc.page_content for doc in docAnimes])
    

    # Construir prompt
    prompt = f"""
    Com base nas informações abaixo, responda à pergunta:

    História inteira de One Piece até o momento: {context1}

    Pergunta: {query}

    Resposta (use markdown para destacar títulos):
    """
    
    # Gerar resposta 
    inputs = tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=2000, temperature=0.7)
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return resposta.split("Resposta:")[-1].strip()

In [4]:
resposta = gerar_respostaComVetorAnime("Qual é a personalidade do personagem Luffy ?")
print(resposta)

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
c:\Anaconda\envs\raganime\Lib\site-packages\transformers\generation\utils.py:2105: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Com base nas informaç�es abaixo, responda à pergunta:

    História inteira de One Piece até o momento: 1   O + V ; i ) :   E   Y e f   1   1   3 2 4 9   w 7   ;   1   8   1
D 9 S   1   ~ K y y h t " "   8   4   H e G a L O -   8 1   1

    Pergunta: Qual é a personalidade do personagem Luffy ?

    Resposta (use markdown para destacar títulos):
    
    **Pessoa desorganizada e apaixonada por comida**

    A base para essa resposta está na primeira palavra da primeira linha, que representa o início da história de One Piece. Essa história conta a jornada de Luffy e seu grupo, conhecidos como a Equipe Straw Hat, para encontrar o "Rei do Mundo dos Maravilhosos", um título atribuído a aquele que possui o "O Tesouro do Mundo", o Maior tesouro da Terra.

    Luffy é o líder e o protagonista da história. Ele tem a habilidade de se expandir e se contrair, além de ser um personagem inovador e aventureiro. Ele é conhecido por ser desorganizado, pois não gosta de organizar suas coisas e preferir